In [ ]:
import requests
import random
from multiprocessing import Pool
import tqdm


In [ ]:
headers = {
  'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0',
  'Accept': '*/*',
  'Accept-Language': 'pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3',
  'X-Requested-With': 'XMLHttpRequest',
  'Origin': 'https://farmrpg.com',
  'Connection': 'keep-alive',
  'Referer': 'https://farmrpg.com/',
  'Cookie': '',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'Content-Length': '0',
  'TE': 'trailers'
}

def make_headers(farmrpg_token, high_wind):
  headers['Cookie'] = f'farmrpg_token={farmrpg_token}; HighwindFRPG={high_wind};'

def login(user, password):
  url = "https://farmrpg.com/worker.php?go=login"

  payload = f"username={user}&password={password}"
  _headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0',
    'Accept': '*/*',
    'Accept-Language': 'pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'https://farmrpg.com',
    'Connection': 'keep-alive',
    'Referer': 'https://farmrpg.com/index.php',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'TE': 'trailers'
  }

  response = requests.request("POST", url, headers=_headers, data=payload)

  cookies = response.headers['set-cookie']
  if response.text == 'success':
    farmrpg_token = cookies.split("farmrpg_token=")[1].split(';')[0]
    high_wind = cookies.split("HighwindFRPG=")[1].split(';')[0]

  make_headers(farmrpg_token, high_wind)

In [ ]:

def get_worms(response):
    return int(response.split("Worms: <strong>")[1].split("</strong>")[0])

def get_explores(response):
    return int(
        (response.split('<div id="explorestam">')[1].split("</div>")[0]).replace(",", "")
    )


In [ ]:
def fishcaught(_):
    # get random number
    r = random.randint(10000, 99999)
    r2 = random.randint(400, 600)

    url = f"https://farmrpg.com/worker.php?go=fishcaught&id=8&r={r}&bamt=1&p=1053&q={r2}"
    payload = {}
    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code != 200:
        print("Error")

def chargebait():
    url = "https://farmrpg.com/worker.php?go=chargebait"
    payload = {}
    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code != 200:
        print("Error")

def buy_worms(quantity):
    url = f"https://farmrpg.com/worker.php?go=buyitem&id=18&qty={quantity}"
    payload = {}
    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code != 200:
        print("Error")

def sellalluserfish():
    url = "https://farmrpg.com/worker.php?go=sellalluserfish"
    payload = {}
    response = requests.request("POST", url, headers=headers, data=payload)

def baitarea(_):
    url = "https://farmrpg.com/worker.php?go=baitarea&id=5"
    payload = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.text

def explore(_):
    url = "https://farmrpg.com/worker.php?go=explore&id=6"
    payload = {}
    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code != 200:
        print("Error")

    return response.text

def get_streak():
    url = "https://farmrpg.com/fishing.php?id=8"
    payload = {}
    response = requests.request("GET", url, headers=headers, data=payload)

    return response.text.split("Streak: <strong>")[1].split("</strong>")[0]

def reset_streak():
    while int(get_streak()) > 0:
        chargebait()

In [ ]:
USER = "USER"
PASSWORD = "PASSWORD"

login(USER, PASSWORD)

In [ ]:
while True:
    response = explore(1)
    explores = get_explores(response)

    if explores <= 0:
        break

    with Pool(5) as p:
        p.map(explore, range(explores))

In [ ]:
for _ in tqdm.tqdm(range(1000)):
    response = baitarea(1)
    worms = get_worms(response)

    if worms <= 0:
        buy_worms(200)
        worms = 200

    with Pool(5) as p:
        p.map(fishcaught, range(worms))

    reset_streak()
    sellalluserfish()